In [42]:
import pandas as pd 
import numpy as np
from dataclasses import dataclass

import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [113]:
def monthly_mortgage_cost(credit_amount,pct,n_months):
    '''
    Калькуляция месячной стоимости ипотеки

    M = P [ i(1 + i)^n ] / [ (1 + i)^n - 1]
    
    '''

    monthly_pct = pct / 12
    ttl_pct = (1+ monthly_pct)**n_months

    up_ = monthly_pct * ttl_pct
    down_ = ttl_pct - 1
    
    monthly_payment = credit_amount * up_ / down_

    return monthly_payment

In [636]:
@dataclass
class RealEstateObject:
    """
    General
    ---
    Class representing of result of mortrage deal

    Attributes:  
    ---- 
        - asset_price - price of household itself
        - first_sum_encounter - initial down payment for mortgage
        - mortgage_rate - bank yearly mortgage rate
        - mortgage_period - initial payment period in months
    
    """
    asset_price:int
    first_sum_encounter:int
    mortgage_rate:float
    mortgage_period:int
    inflation_rate:float = 0.04

    def __post_init__(self):

        self.credit_amount = self.asset_price - self.first_sum_encounter

        self.monthly_payment = monthly_mortgage_cost(self.credit_amount,self.mortgage_rate,self.mortgage_period).__round__()

        self.asset_price_npv = int(self.asset_price * (1 + self.inflation_rate/12)**self.mortgage_period)

        self.asset_total_cost = int(self.monthly_payment * self.mortgage_period)
             
        self.profit =  self.asset_price_npv - self.asset_total_cost

        return self

    def custom_mortgage_calc(self,early_payments_period:int,
                                    early_payments_amount:int ):

        total_bonus_payments = 0
        credit_amount = self.credit_amount

        for nth_month in range(1,self.mortgage_period+1)[::-1]:
            
            if nth_month % early_payments_period == 0:
                credit_amount -= early_payments_amount
                total_bonus_payments += early_payments_amount

            mpay = monthly_mortgage_cost(credit_amount+early_payments_amount,self.mortgage_rate,nth_month)

            if credit_amount <= mpay:
                break
            
            credit_amount -= mpay
            

        self.mortgage_period = self.mortgage_period -  nth_month
        self.asset_total_cost = int(self.monthly_payment * self.mortgage_period) + total_bonus_payments
        self.profit =  self.asset_price_npv - self.asset_total_cost

        return self



In [663]:
base_terms = {
    'asset_price':17_000_000,
    'first_sum_encounter':2_550_000,
    'mortgage_rate':.109,
    'mortgage_period':20*12,
}
sber_terms = {
    'asset_price':17_000_000,
    'first_sum_encounter':2_000_000,
    'mortgage_rate':.079,
    'mortgage_period':20*12,
}
it_terms = {
    'asset_price':17_000_000,
    'first_sum_encounter':17_000_000 * .15,
    'mortgage_rate':.057,
    'mortgage_period':20*12,
}

In [664]:
base_mort = RealEstateObject(**base_terms)
sber_mort = RealEstateObject(**sber_terms)
it_mort = RealEstateObject(**it_terms)

In [665]:
base_mort.__dict__

{'asset_price': 17000000,
 'first_sum_encounter': 2550000,
 'mortgage_rate': 0.109,
 'mortgage_period': 240,
 'inflation_rate': 0.04,
 'credit_amount': 14450000,
 'monthly_payment': 148169,
 'asset_price_npv': 37783895,
 'asset_total_cost': 35560560,
 'profit': 2223335}

In [742]:
base_mort.custom_mortgage_calc(12,100_000)

RealEstateObject(asset_price=17000000, first_sum_encounter=2550000, mortgage_rate=0.109, mortgage_period=216, inflation_rate=0.04)

In [743]:
base_mort.__dict__

{'asset_price': 17000000,
 'first_sum_encounter': 2550000,
 'mortgage_rate': 0.109,
 'mortgage_period': 216,
 'inflation_rate': 0.04,
 'credit_amount': 14450000,
 'monthly_payment': 148169,
 'asset_price_npv': 37783895,
 'asset_total_cost': 33904504,
 'profit': 3879391}

In [666]:
sber_mort.__dict__

{'asset_price': 17000000,
 'first_sum_encounter': 2000000,
 'mortgage_rate': 0.079,
 'mortgage_period': 240,
 'inflation_rate': 0.04,
 'credit_amount': 15000000,
 'monthly_payment': 124534,
 'asset_price_npv': 37783895,
 'asset_total_cost': 29888160,
 'profit': 7895735}

In [667]:
it_mort.__dict__

{'asset_price': 17000000,
 'first_sum_encounter': 2550000.0,
 'mortgage_rate': 0.057,
 'mortgage_period': 240,
 'inflation_rate': 0.04,
 'credit_amount': 14450000.0,
 'monthly_payment': 101039,
 'asset_price_npv': 37783895,
 'asset_total_cost': 24249360,
 'profit': 13534535}